In [ ]:
# --- Cell 1: Fix TensorFlow / protobuf mismatch, clean restart ---
!pip uninstall -y protobuf -q
!pip install -q --no-deps "protobuf==3.20.*"

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"   # hide INFO & WARNING logs
os.environ["PYTHONHASHSEED"] = "42"

import os as _os
_os.kill(_os.getpid(), 9)   # restart kernel (expected behaviour)


In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing import image
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, Model
from keras.regularizers import l2
from tensorflow.keras.layers import (
    Activation, Dropout, Dense, Flatten, Conv2D, BatchNormalization,
    MaxPooling2D, GlobalAveragePooling2D, Input, concatenate, Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import warnings
warnings.filterwarnings("ignore")

print("✅ Imports successful — TensorFlow version:", tf.__version__)


In [ ]:
# Main Folder Path
folder_path = "/kaggle/input/kermany2018/OCT2017 "

# Sub Folder Paths
train_dir = f"{folder_path}/train"
val_dir = f"{folder_path}/val"
test_dir = f"{folder_path}/test"

In [ ]:
os.listdir(folder_path)


In [ ]:
print(f"Train Directory: {os.listdir(train_dir)}")
print(f"Validation Directory: {os.listdir(test_dir)}")
print(f"Test Directory: {os.listdir(val_dir)}")

In [ ]:
normal_train_dir = os.path.join(train_dir, "NORMAL")
normal_train_files = os.listdir(normal_train_dir)[:30]

normal_train_files

In [ ]:
normal_train_files[17]


In [ ]:
# Image file path
image_file = "NORMAL/NORMAL-8869683-18.jpeg"
image_path = os.path.join(train_dir, image_file)

# Read and display the image
image = Image.open(image_path)
plt.imshow(image)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
# Specify the directory where the dataset is located
dataset_directory = train_dir

# Create a dictionary to store the counts of images for each class
image_counts = {"CNV": 0, "DME": 0, "DRUSEN": 0, "NORMAL": 0}

# Iterate through the dataset to count the number of images for each class
for class_name in image_counts.keys():
    class_directory = os.path.join(dataset_directory, class_name)
    image_counts[class_name] = len(os.listdir(class_directory))

# Plotting the graph
classes = list(image_counts.keys())
counts = list(image_counts.values())

fig, ax = plt.subplots()
bars = ax.bar(classes, counts, color=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])

# Display total counts above the bars
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 0.1, round(yval), ha='center', va='bottom')

plt.xlabel('Classes')
plt.ylabel('Number of Images')
plt.title('Number of Images in Train Classes')
plt.show()

In [ ]:
import os, random, math, shutil
from pathlib import Path
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd

random.seed(42)

# --- Directories ---
src = Path(test_dir)                         # existing test directory
dst = Path("/kaggle/working/test_ratio")     # new ratio-matched test directory

# --- Classes & Target Ratios (from your train set) ---
classes = ["CNV", "DME", "DRUSEN", "NORMAL"]
ratios  = {"CNV":0.446, "DME":0.136, "DRUSEN":0.103, "NORMAL":0.315}

# 1️⃣ Count how many images exist per class in the source
avail = {c: len(list((src/c).glob("*"))) for c in classes}
print("Available per class:", avail, "Total:", sum(avail.values()))

# 2️⃣ Compute the largest feasible total possible given class availability
feasible_total = min(math.floor(avail[c] / ratios[c]) for c in classes)
print("Largest feasible total:", feasible_total)

# 3️⃣ Compute desired samples per class based on ratios
desired = {c: int(round(ratios[c] * feasible_total)) for c in classes}
# clip by availability just in case
for c in classes:
    desired[c] = min(desired[c], avail[c])

# adjust if needed so sum matches feasible_total
diff = feasible_total - sum(desired.values())
if diff != 0:
    order = sorted(classes, key=lambda c: (avail[c]-desired[c]), reverse=True) if diff > 0 \
            else sorted(classes, key=lambda c: desired[c], reverse=True)
    i = 0
    while diff != 0 and i < len(order):
        c = order[i]
        if diff > 0 and desired[c] < avail[c]:
            desired[c] += 1; diff -= 1
        elif diff < 0 and desired[c] > 0:
            desired[c] -= 1; diff += 1
        else:
            i += 1

print("Desired per class:", desired, "Total:", sum(desired.values()))

# 4️⃣ Sample and copy files into new folder
dst.mkdir(parents=True, exist_ok=True)
rows = []
for c in classes:
    files = [p for p in (src/c).glob("*")]
    pick  = random.sample(files, desired[c])
    (dst/c).mkdir(parents=True, exist_ok=True)
    for p in pick:
        shutil.copy2(p, dst/c/p.name)
        rows.append({"filepath": str(dst/c/p.name), "label": c})

df = pd.DataFrame(rows)
print(f"✅ Created {len(df)} files under {dst}")

# 5️⃣ Plot class distribution with your color scheme
counts = dict(Counter(df["label"]))
fig, ax = plt.subplots()
bars = ax.bar(classes, [counts[c] for c in classes],
              color=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])
for bar in bars:
    yval = int(bar.get_height())
    ax.text(bar.get_x() + bar.get_width()/2, yval + 0.1, yval, ha='center', va='bottom')
ax.set_xlabel('Classes')
ax.set_ylabel('Number of Images')
ax.set_title('Number of Images in Ratio-Matched Test Set')
plt.show()


In [ ]:
import os, cv2
import numpy as np

dataset_directory = train_dir
classes = ["CNV", "DME", "DRUSEN", "NORMAL"]

TARGET_SIZE = (299, 299)  # for InceptionV3

def first_image_path(folder):
    for fname in os.listdir(folder):
        if fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")):
            return os.path.join(folder, fname)
    return None

for cls in classes:
    cls_dir = os.path.join(dataset_directory, cls)
    path = first_image_path(cls_dir)
    if path is None:
        print(f"Class: {cls}, no image files found")
        continue

    img_bgr = cv2.imread(path, cv2.IMREAD_COLOR)  # ensure 3 channels
    if img_bgr is None:
        print(f"Class: {cls}, failed to read: {path}")
        continue

    # Convert BGR->RGB for consistency with Keras/TensorFlow
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # Report shapes
    orig_shape = img_rgb.shape  # (H, W, 3)
    resized = cv2.resize(img_rgb, TARGET_SIZE, interpolation=cv2.INTER_AREA)
    resized_shape = resized.shape

    print(f"Class: {cls}, Original: {orig_shape}, Resized: {resized_shape}")


In [ ]:
import random
random.seed(42)
path = os.path.join(cls_dir, random.choice(os.listdir(cls_dir)))


In [ ]:
import os, random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread

random.seed(42)

classes = ["CNV", "DME", "DRUSEN", "NORMAL"]
root = Path(train_dir)

fig, axs = plt.subplots(1, 4, figsize=(12, 3))

for i, cls in enumerate(classes):
    cls_dir = root / cls
    # pick a random image file (basic extensions)
    files = [p for p in cls_dir.iterdir() if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}]
    if not files:
        axs[i].set_title(f"{cls}\n(no images)"); axs[i].axis("off"); continue
    img_path = random.choice(files)

    img = imread(img_path)

    # if grayscale, stack to RGB for consistent display
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)

    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(cls)

plt.tight_layout()
plt.show()


In [ ]:
batch_size = 32


In [ ]:
image_shape = (256,256,1)


In [ ]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.20, # Shift the pic width by a max of 20%
                               height_shift_range=0.15, # Shift the pic height by a max of 15%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.15, # Shear means cutting away part of the image (max 15%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
image_gen.flow_from_directory(train_dir)


In [ ]:
image_gen.flow_from_directory(test_dir)


In [ ]:
from tensorflow.keras import Sequential, layers, regularizers

image_shape = (256, 256, 1)   # grayscale
num_classes = 4

model = Sequential([
    layers.Input(shape=image_shape),

    layers.Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Dropout(0.10),

    layers.Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Dropout(0.15),

    layers.Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Dropout(0.20),

    layers.Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Dropout(0.20),

    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu',
                 kernel_initializer='he_normal',
                 kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.30),
    layers.Dense(num_classes, activation='softmax')
])


In [ ]:
from tensorflow.keras import optimizers
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(1e-3),
              metrics=['accuracy'])


In [ ]:
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=1)
ckpt       = ModelCheckpoint('best_cnn.h5', monitor='val_accuracy', save_best_only=True)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 32
IMG_SIZE   = (256, 256)

image_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True
)

train_image_gen = image_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)


In [ ]:
val_test_gen = ImageDataGenerator(rescale=1./255)

test_image_gen = val_test_gen.flow_from_directory(
    test_dir,            # use val_dir here if you prefer a separate validation split
    target_size=IMG_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
results = model.fit(
    train_image_gen,
    epochs=10,
    validation_data=test_image_gen,
    callbacks=[early_stop, reduce_lr, ckpt],
    verbose=1
)


In [ ]:
summary = pd.DataFrame(results.history)
summary.head()


In [ ]:
results.history


In [ ]:
history_df = pd.DataFrame(results.history)
history_df.index.name = 'Epoch'
history_df.index += 1
print(history_df)


In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary['loss'],     label="loss")
plt.plot(summary['val_loss'], label="val_loss")
plt.legend(loc="upper right"); plt.ylabel("Loss"); plt.xlabel("Epoch")
plt.title("Training vs Validation Loss")
plt.show()


In [ ]:
model.evaluate(test_image_gen, verbose=1)


In [ ]:
model.metrics_names


In [ ]:
pred_probabilities = model.predict(test_image_gen, verbose=1)


In [ ]:
pred_probabilities.shape


In [ ]:
y_true = test_image_gen.classes
y_true[:10]


In [ ]:
predictions = pred_probabilities


In [ ]:
from sklearn.metrics import classification_report
y_pred = np.argmax(predictions, axis=1)
target_names = list(test_image_gen.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))


In [ ]:
model.save('CNN_model1.h5')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE   = (299, 299)   # InceptionV3 native size
BATCH_SIZE = 32

# Generators: scale to [0,1] only (we'll map to [-1,1] in the model)
iv3_train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True
)
iv3_eval_aug = ImageDataGenerator(rescale=1./255)


In [ ]:
iv3_train_gen = iv3_train_aug.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)


In [ ]:
iv3_val_gen = iv3_eval_aug.flow_from_directory(
    test_dir,                    # use val_dir here if you have one
    target_size=IMG_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
x_batch, y_batch = next(iv3_train_gen)
print("Batch shape:", x_batch.shape)  # expect (B, 299, 299, 1)
assert x_batch.shape[1:] == (299, 299, 1), "Generator shape is not 299x299x1 — recreate generators."


In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, layers, optimizers
from tensorflow.keras.applications import InceptionV3

num_classes = iv3_train_gen.num_classes

inputs = layers.Input(shape=(299, 299, 1), name='gray_input')
x = layers.Concatenate(name='gray_to_rgb')([inputs, inputs, inputs])      # (299,299,3)
x = layers.Rescaling(scale=2.0, offset=-1.0, name='to_minus1_plus1')(x)   # [0,1] -> [-1,1]

base = InceptionV3(weights='imagenet', include_top=False)
base.trainable = False

x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

iv3_model = Model(inputs, outputs, name='InceptionV3_OCT')


In [ ]:
iv3_model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(1e-4),
    metrics=['accuracy']
)


In [ ]:
iv3_model.summary()


In [ ]:
import os
cls2idx = iv3_train_gen.class_indices
train_counts = {c: len(os.listdir(os.path.join(train_dir, c))) for c in cls2idx}
total = sum(train_counts.values())
class_weights = {cls2idx[c]: total/(len(cls2idx)*train_counts[c]) for c in cls2idx}
class_weights


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

ckpt_path = 'InceptionV3_tuning.keras'
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=1)
checkpoint = ModelCheckpoint(ckpt_path, monitor='val_loss', save_best_only=True, verbose=1)
csv_log    = CSVLogger('training_log.csv', append=False)


In [ ]:
history_s1 = iv3_model.fit(
    iv3_train_gen,
    validation_data=iv3_val_gen,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr, checkpoint, csv_log],
    verbose=1
)


In [ ]:
import pandas as pd, matplotlib.pyplot as plt

s1 = pd.DataFrame(history_s1.history); s1.index += 1

plt.figure(figsize=(10,6)); plt.plot(s1['loss'], label='loss'); plt.plot(s1['val_loss'], label='val_loss')
plt.legend(); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Stage 1 Loss'); plt.show()

plt.figure(figsize=(10,6)); plt.plot(s1['accuracy'], label='acc'); plt.plot(s1['val_accuracy'], label='val_acc')
plt.legend(); plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Stage 1 Accuracy'); plt.show()


In [ ]:
for layer in base.layers[-50:]:
    layer.trainable = True

iv3_model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(1e-5),
    metrics=['accuracy']
)


In [ ]:
history_s2 = iv3_model.fit(
    iv3_train_gen,
    validation_data=iv3_val_gen,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr, checkpoint, csv_log],
    verbose=1
)


In [ ]:
loss, acc = iv3_model.evaluate(iv3_val_gen, verbose=1)
print(f"Test — loss: {loss:.4f}  acc: {acc:.4f}")


In [ ]:
pred_prob = iv3_model.predict(iv3_val_gen, verbose=1)
pred_prob.shape


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = iv3_val_gen.classes
y_pred = np.argmax(pred_prob, axis=1)
target_names = list(iv3_val_gen.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted'); plt.ylabel('True'); plt.title('Confusion Matrix — InceptionV3'); plt.show()


In [ ]:
iv3_model.save('InceptionV3_final.keras')


In [ ]:
# === Column 0: Fix TF/protobuf (run ONCE, before importing tensorflow) ===
!pip uninstall -y protobuf -q
!pip install -q --no-deps "protobuf==3.20.3"

import os
os.kill(os.getpid(), 9)  # hard-restart kernel so TF picks up the right protobuf


In [ ]:
# === Column 1: Setup & robust OCT2017 dataset resolver ===
import os, warnings, random, zipfile, glob
from pathlib import Path
import numpy as np
import tensorflow as tf

# Quiet logs + seeds
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

print("TF:", tf.__version__)
print("Mounted roots under /kaggle/input:", [p.name for p in Path("/kaggle/input").iterdir()])

def _has_splits(p: Path):
    return all((p/s).is_dir() for s in ("train","val","test"))

def find_or_extract_oct():
    root = Path("/kaggle/input")

    # 1) Look for an OCT2017 directory anywhere under /kaggle/input
    for p in root.rglob("OCT2017"):
        if _has_splits(p):
            return p

    # 2) Fallback: find a parent that has train/val/test with OCT class folders
    for tr in root.rglob("train"):
        if all((tr/c).is_dir() for c in ("CNV","DME","DRUSEN","NORMAL")):
            parent = tr.parent
            if _has_splits(parent):
                return parent

    # 3) Last resort: extract any OCT* zip to /kaggle/working and search again
    zips = [z for z in root.rglob("*.zip") if "OCT" in z.name.upper()]
    if zips:
        out_dir = Path("/kaggle/working/oct2017_extracted")
        out_dir.mkdir(parents=True, exist_ok=True)
        print("Extracting:", zips[0])
        with zipfile.ZipFile(zips[0]) as zf:
            zf.extractall(out_dir)
        for p in out_dir.rglob("OCT2017"):
            if _has_splits(p):
                return p

    return None

oct_root = find_or_extract_oct()
assert oct_root is not None, (
    "OCT2017 not found. In the right panel, add dataset 'paultimothymooney/kermany2018'."
)

train_dir = str(oct_root / "train")
val_dir   = str(oct_root / "val")
test_dir  = str(oct_root / "test")

print("Using OCT2017 at:", str(oct_root))
for name, p in (("Train",train_dir), ("Val",val_dir), ("Test",test_dir)):
    print(f" - {name}: {p} —", "OK" if Path(p).is_dir() else "MISSING")

classes = sorted([d.name for d in Path(train_dir).iterdir() if d.is_dir()])
print("Classes:", classes)


In [ ]:
# === Column 2: Data generators (EffNetB3) + class weights ===
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

IMG_SIZE   = (300, 300)     # EfficientNetB3 native
BATCH_SIZE = 32

# Augment only on training; no rescale/preprocess here
train_aug = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True
)
eval_aug = ImageDataGenerator()

train_gen = train_aug.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)
val_gen = eval_aug.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)
test_gen = eval_aug.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# Sanity check: expect (batch, 300, 300, 3)
xb, yb = next(train_gen)
print("Train batch:", xb.shape, yb.shape)
print("Class indices:", train_gen.class_indices)

# Class weights for imbalance: total / (n_classes * count_c)
y_labels = train_gen.classes
n_cls = len(train_gen.class_indices)
counts = np.bincount(y_labels, minlength=n_cls)
class_weights = {i: (y_labels.size / (n_cls * c)) for i, c in enumerate(counts)}
print("Counts:", counts.tolist())
print("Class weights:", class_weights)


In [ ]:
# === Column 3: EffNetB3 model + 2-stage training ===
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# 3.1 Build model (pretrained; preprocessing is inside EfficientNetB3)
inputs = tf.keras.Input(shape=(*IMG_SIZE, 3))
base = tf.keras.applications.EfficientNetB3(include_top=False, weights='imagenet', input_tensor=inputs)
x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dropout(0.30)(x)
outputs = layers.Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs, outputs, name="EffNetB3_OCT")
model.summary()

# 3.2 Callbacks
ckpt = ModelCheckpoint('EffNetB3.best.keras', monitor='val_loss', save_best_only=True, verbose=1)
early = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
rlr  = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.3, verbose=1)
csv  = CSVLogger('training.log.csv', append=False)

# 3.3 Stage 1: freeze backbone, train head
for l in base.layers:  # freeze all
    l.trainable = False

model.compile(optimizer=optimizers.Adam(1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_s1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early, rlr, ckpt, csv],
    verbose=1
)

# 3.4 Stage 2: fine-tune top layers (keep BatchNorm frozen)
for l in base.layers:
    if isinstance(l, tf.keras.layers.BatchNormalization):
        l.trainable = False
for l in base.layers[-200:]:
    if not isinstance(l, tf.keras.layers.BatchNormalization):
        l.trainable = True

model.compile(optimizer=optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_s2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early, rlr, ckpt],
    verbose=1
)

# 3.5 Evaluate and report
val_loss, val_acc   = model.evaluate(val_gen, verbose=1)
test_loss, test_acc = model.evaluate(test_gen, verbose=1)
print(f"Val  : loss={val_loss:.4f} acc={val_acc:.4f}")
print(f"Test : loss={test_loss:.4f} acc={test_acc:.4f}")

# 3.6 Classification report + confusion matrix on VAL
pred_prob = model.predict(val_gen, verbose=1)
y_true = val_gen.classes
y_pred = pred_prob.argmax(axis=1)
target_names = list(val_gen.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=target_names))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix — EffNetB3 (val)")
plt.show()

# 3.7 Save final model
model.save('EffNetB3.final.keras')


In [ ]:
# === Column 4: Learning curves + PR/ROC + Test report ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from sklearn.metrics import (classification_report, confusion_matrix,
                             precision_recall_curve, average_precision_score,
                             roc_curve, auc)
from sklearn.preprocessing import label_binarize
from pathlib import Path

Path("/kaggle/working").mkdir(parents=True, exist_ok=True)

# 4.1 Combine histories and plot
def combine_histories(h1, h2):
    df1 = pd.DataFrame(h1.history)[["loss","val_loss","accuracy","val_accuracy"]]; df1.index += 1
    df2 = pd.DataFrame(h2.history)[["loss","val_loss","accuracy","val_accuracy"]]; df2.index += len(df1) + 1
    return pd.concat([df1, df2], axis=0)

hist_df = combine_histories(history_s1, history_s2)
hist_df.to_csv("/kaggle/working/history_combined.csv")

plt.figure(figsize=(8,5))
plt.plot(hist_df.index, hist_df["loss"], label="loss")
plt.plot(hist_df.index, hist_df["val_loss"], label="val_loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Training vs Validation Loss")
plt.legend(); plt.tight_layout(); plt.savefig("/kaggle/working/curve_loss.png"); plt.show()

plt.figure(figsize=(8,5))
plt.plot(hist_df.index, hist_df["accuracy"], label="acc")
plt.plot(hist_df.index, hist_df["val_accuracy"], label="val_acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.title("Training vs Validation Accuracy")
plt.legend(); plt.tight_layout(); plt.savefig("/kaggle/working/curve_accuracy.png"); plt.show()

# 4.2 Evaluate on VAL and TEST
val_loss, val_acc = model.evaluate(val_gen, verbose=1)
test_loss, test_acc = model.evaluate(test_gen, verbose=1)
print(f"VAL  -> loss {val_loss:.4f}  acc {val_acc:.4f}")
print(f"TEST -> loss {test_loss:.4f}  acc {test_acc:.4f}")

# 4.3 Reports + Confusion Matrix (TEST)
pred_prob_test = model.predict(test_gen, verbose=1)
y_true_test = test_gen.classes
y_pred_test = pred_prob_test.argmax(axis=1)
target_names = list(test_gen.class_indices.keys())

print(classification_report(y_true_test, y_pred_test, target_names=target_names))

cm = confusion_matrix(y_true_test, y_pred_test)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix — Test")
plt.tight_layout(); plt.savefig("/kaggle/working/cm_test.png"); plt.show()

# 4.4 Precision–Recall curves (TEST)
n_classes = len(target_names)
y_true_bin = label_binarize(y_true_test, classes=list(range(n_classes)))

plt.figure(figsize=(7,5))
aps = []
for i, name in enumerate(target_names):
    pr, rc, _ = precision_recall_curve(y_true_bin[:, i], pred_prob_test[:, i])
    ap = average_precision_score(y_true_bin[:, i], pred_prob_test[:, i])
    aps.append(ap)
    plt.plot(rc, pr, label=f"{name} (AP={ap:.3f})")
# micro-average
pr_micro, rc_micro, _ = precision_recall_curve(y_true_bin.ravel(), pred_prob_test.ravel())
ap_micro = average_precision_score(y_true_bin, pred_prob_test, average="micro")
plt.plot(rc_micro, pr_micro, linestyle="--", label=f"micro-avg (AP={ap_micro:.3f})")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision–Recall (Test)")
plt.legend(); plt.tight_layout(); plt.savefig("/kaggle/working/pr_curves.png"); plt.show()
print(f"Per-class AP: {dict(zip(target_names, [float(a) for a in aps]))}")
print(f"Micro-avg AP: {ap_micro:.3f}")

# 4.5 ROC curves (TEST)
plt.figure(figsize=(7,5))
aucs = []
for i, name in enumerate(target_names):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], pred_prob_test[:, i])
    roc_auc = auc(fpr, tpr); aucs.append(roc_auc)
    plt.plot(fpr, tpr, label=f"{name} (AUC={roc_auc:.3f})")
# micro-average
fpr_micro, tpr_micro, _ = roc_curve(y_true_bin.ravel(), pred_prob_test.ravel())
auc_micro = auc(fpr_micro, tpr_micro)
plt.plot(fpr_micro, tpr_micro, linestyle="--", label=f"micro-avg (AUC={auc_micro:.3f})")
plt.plot([0,1],[0,1], linestyle=":")  # chance
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC (Test)")
plt.legend(); plt.tight_layout(); plt.savefig("/kaggle/working/roc_curves.png"); plt.show()
print(f"Per-class AUC: {dict(zip(target_names, [float(a) for a in aucs]))}")
print(f"Micro-avg AUC: {auc_micro:.3f}")
